In [1]:
import sqlite3
import random
from datetime import datetime

In [2]:
# Establish a connection
conn = sqlite3.connect("chemical_plant_monitoring.db")

# Create cursor
cursor = conn.cursor()

In [3]:
# Create tables 
cursor.execute("""
CREATE TABLE IF NOT EXISTS plants (
    plant_id INTEGER PRIMARY KEY,
    name TEXT,
    location TEXT
);
""")

In [4]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS products (
    product_id INTEGER PRIMARY KEY,
    product_name TEXT,
    hazard_level TEXT
);
""")

In [5]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS production (
    production_id INTEGER PRIMARY KEY,
    plant_id INTEGER,
    product_id INTEGER,
    year INTEGER,
    month INTEGER,
    quantity_produced_kg REAL,
    FOREIGN KEY(plant_id) REFERENCES plants(plant_id),
    FOREIGN KEY(product_id) REFERENCES products(product_id)
);
""")

In [6]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS emissions (
    emission_id INTEGER PRIMARY KEY,
    plant_id INTEGER,
    year INTEGER,
    month INTEGER,
    CO2_tons REAL,
    NOx_tons REAL,
    SO2_tons REAL,
    FOREIGN KEY(plant_id) REFERENCES plants(plant_id)
);
""")

In [7]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS energy_usage (
    energy_id INTEGER PRIMARY KEY,
    plant_id INTEGER,
    year INTEGER,
    month INTEGER,
    total_energy_MWh REAL,
    renewable_pct REAL,
    FOREIGN KEY(plant_id) REFERENCES plants(plant_id)
);
""")

In [8]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS incidents (
    incident_id INTEGER PRIMARY KEY,
    plant_id INTEGER,
    year INTEGER,
    month INTEGER,
    type TEXT,
    severity TEXT,
    FOREIGN KEY(plant_id) REFERENCES plants(plant_id)
);
""")

In [9]:
# Commit the changes
conn.commit()

In [10]:
# Simulated Data
plants = [
    (1, "Plant Alpha", "Brazil"),
    (2, "Plant Beta", "USA"),
    (3, "Plant Gamma", "Germany")
]

products = [
    (1, "Sodium Hydroxide", "Medium"),
    (2, "Ammonia", "High"),
    (3, "Hydrochloric Acid", "High"),
    (4, "Ethanol", "Low")
]

In [11]:
# Insert Data

cursor.executemany("""
INSERT OR IGNORE INTO plants (plant_id, name, location) VALUES (?, ?, ?)
""", plants)

cursor.executemany("""
INSERT OR IGNORE INTO products (product_id, product_name, hazard_level) VALUES (?, ?, ?)
""", products)

In [12]:
# Generate Data

def generate_production_data():
    for year in range(2021, 2025):
        for month in range(1, 13):
            for plant_id in range(1, 4):
                for product_id in range(1, 5):
                    qty = round(random.uniform(1000, 50000), 2)
                    cursor.execute("""
                    INSERT INTO production (plant_id, product_id, year, month, quantity_produced_kg)
                    VALUES (?, ?, ?, ?, ?)
                    """, (plant_id, product_id, year, month, qty))

In [13]:
def generate_emissions_data():
    for year in range(2021, 2025):
        for month in range(1, 13):
            for plant_id in range(1, 4):
                co2 = round(random.uniform(100, 1000), 2)
                nox = round(random.uniform(10, 100), 2)
                so2 = round(random.uniform(5, 50), 2)
                cursor.execute("""
                INSERT INTO emissions (plant_id, year, month, CO2_tons, NOx_tons, SO2_tons)
                VALUES (?, ?, ?, ?, ?, ?)
                """, (plant_id, year, month, co2, nox, so2))

In [14]:
def generate_energy_data():
    for year in range(2021, 2025):
        for month in range(1, 13):
            for plant_id in range(1, 4):
                energy = round(random.uniform(500, 5000), 2)
                renewable = round(random.uniform(10, 90), 2)
                cursor.execute("""
                INSERT INTO energy_usage (plant_id, year, month, total_energy_MWh, renewable_pct)
                VALUES (?, ?, ?, ?, ?)
                """, (plant_id, year, month, energy, renewable))

In [15]:
def generate_incident_data():
    types = ["Spill", "Fire", "Leak", "Power Outage"]
    severities = ["Low", "Medium", "High", "Critical"]
    for year in range(2021, 2025):
        for month in range(1, 13):
            for plant_id in range(1, 4):
                if random.random() < 0.3:  # 30% chance de incidente no mês
                    for _ in range(random.randint(0, 2)):
                        incident_type = random.choice(types)
                        severity = random.choices(severities, weights=[40, 30, 20, 10])[0]
                        cursor.execute("""
                        INSERT INTO incidents (plant_id, year, month, type, severity)
                        VALUES (?, ?, ?, ?, ?)
                        """, (plant_id, year, month, incident_type, severity))


In [16]:
# Execute Data generation

generate_production_data()
generate_emissions_data()
generate_energy_data()
generate_incident_data()

conn.commit()
conn.close()